# Problema das Rainhas do Jogo de Xadrez

## Problema

Em um tabuleiro de xadrez (padrão 8x8) vazio, sabendo-se que uma alocação em casa preta vale o dobro do que em casa branca, determine a localização ótima para 8 rainhas de modo que nenhuma delas seja ameaçada pelas demais.

<hr style="height:2px; background-color:gray; border:none;">

## Modelagem matemática

### Conjuntos

Linhas $I = \{1,2,3,4,5,6,7,8\}$<br>
Colunas $J = \{1,2,3,4,5,6,7,8\}$<br>
_Nota_: Apesar de as colunas serem tradicionalmente representadas por letras no xadrez, aqui representaremos como números para facilitar a modelagem do problema.

### Parâmetros

$w_{ij}$: valor da casa $(i,j)$,<br>
onde $$w_{ij} = \begin{cases}
                    2, & \text{se a casa (i,j) é preta} \\\\
                    1, & \text{se a casa (i,j) é branca}
                \end{cases}$$
<br>
Além disso, vamos definir:
- $i+j$ par -> casa preta
- $i+j$ ímpar -> casa branca

### Variáveis de decisão

$x_{ij} \in {0,1}$:<br>
$$x_{ij} = \begin{cases}
                1, & \text{se há uma rainha na casa (i,j)} \\\\
                0, & \text{caso contrário}
            \end{cases}$$

### Função objetivo

Maximizar o valor total das casas ocupadas:<br>
$$ \max Z = \sum\limits_{i \in I} \sum\limits_{j \in J} w_{ij} \cdot x_{ij}$$

### Restrições

##### 1) Exatamente uma rainha por linha

Para todo $i \in I$:<br>
$$ \sum\limits_{j \in J} x_{ij} = 1$$

##### 2) Exatamente uma rainha por coluna

Para todo $j \in J$:<br>
$$ \sum\limits_{i \in I} x_{ij} = 1$$

_Nota_: essas duas primeiras restrições já garantem 8 rainhas no tabuleiro.

##### 3) No máximo uma rainha em cada diagonal principal

Diagonais principais são aquelas com $j - i$ constante.<br>
Definindo o conjunto de diagonais principais como:<br>
$$ D_1 = \{-7, -6, \dots, 6, 7\} $$
Então para cada $ d \in D_1 $:<br>
$$ \sum\limits_{\substack{i \in I, j \in J \\ j-i=d}} x_{ij} \leq 1 $$

##### 4) No máximo uma rainha em cada diagonal secundária

Diagonais secundárias são aquelas com $i - j$ constante.<br>
Definindo o conjunto de diagonais secundárias como:<br>
$$ D_2 = \{2, 3, \dots, 15, 16\} $$
Então para cada $ s \in D_2 $:<br>
$$ \sum\limits_{\substack{i \in I, j \in J \\ i+j=s}} x_{ij} \leq 1 $$

##### 5) Integralidade

$$ x_{ij} \in \{0,1\}, \quad \forall i \in I, j \in J $$

Esse é um caso de **problema de programação linear inteira binária**.

<hr style="height:2px; background-color:gray; border:none;">

## Modelagem e solução computacional

Agora vamos implementar a modelagem feita utilizando a biblioteca pyomo, e calcular a solução do problema.

In [1]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

In [2]:
# 1) Conjuntos
model.I = pyo.RangeSet(1,8)
model.J = pyo.RangeSet(1,8)

In [3]:
# 2) Parâmetros
def weight_init(m,i,j):
    if (i+j) % 2 == 0:
        return 2.0  #casa preta
    else:
        return 1.0  #casa branca
    
model.w = pyo.Param(model.I, model.J, initialize=weight_init)

In [4]:
# 3) Variáveis de Decisão
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)

In [5]:
# 4) Função Objetivo
def obj_rule(m):
    return sum(m.w[i, j] * m.x[i, j] for i in m.I for j in m.J)

model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

In [6]:
# 5) Restrições
def linha_rule(m, i):
    return sum(m.x[i, j] for j in m.J) == 1

model.Linha = pyo.Constraint(model.I, rule=linha_rule)


def coluna_rule(m, j):
    return sum(m.x[i, j] for i in m.I) == 1

model.Coluna = pyo.Constraint(model.J, rule=coluna_rule)


model.D1 = pyo.RangeSet(-7,7)
def d1_rule(m, d):
    return sum(m.x[i,j] for i in m.I for j in m.J if (j-i) == d) <= 1

model.DiagPrincipal = pyo.Constraint(model.D1, rule=d1_rule)


model.D2 = pyo.RangeSet(2,16)
def d2_rule(m, d):
    return sum(m.x[i,j] for i in m.I for j in m.J if (i+j) == d) <= 1

model.DiagSecundaria = pyo.Constraint(model.D2, rule=d2_rule)

In [7]:
# 6) Solver
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\rlmou\AppData\Local\Temp\tmp6rpj9x27.glpk.raw --wglp C:\Users\rlmou\AppData\Local\Temp\tmp5bwrvitf.glpk.glp
 --cpxlp C:\Users\rlmou\AppData\Local\Temp\tmph5xaq1c2.pyomo.lp
Reading problem data from 'C:\Users\rlmou\AppData\Local\Temp\tmph5xaq1c2.pyomo.lp'...
C:\Users\rlmou\AppData\Local\Temp\tmph5xaq1c2.pyomo.lp:532: warning: lower bound of variable 'x2' redefined
C:\Users\rlmou\AppData\Local\Temp\tmph5xaq1c2.pyomo.lp:532: warning: upper bound of variable 'x2' redefined
46 rows, 64 columns, 256 non-zeros
64 integer variables, all of which are binary
596 lines were read
Writing problem data to 'C:\Users\rlmou\AppData\Local\Temp\tmp5bwrvitf.glpk.glp'...
479 lines were written
GLPK Integer Optimizer 5.0
46 rows, 64 columns, 256 non-zeros
64 integer variables, all of which are binary
Preprocessing...
42 rows, 64 columns, 252 non-zeros
64 integer variables, all of which are binary
Scaling...
 A: min|

In [8]:
# 7) Resultados
from pyomo.environ import value

valor_otimo = value(model.Obj)
print(f"Valor total máximo das casas ocupadas: {valor_otimo:.2f}")

Valor total máximo das casas ocupadas: 12.00


In [9]:
print("\nPosições das rainhas (linha, coluna):")
for i in model.I:
    for j in model.J:
        if pyo.value(model.x[i, j]) > 0.5:
            cor = "preta" if (i + j) % 2 == 0 else "branca"
            print(f" - Rainha em ({i}, {j}) na casa {cor}")


Posições das rainhas (linha, coluna):
 - Rainha em (1, 7) na casa preta
 - Rainha em (2, 4) na casa preta
 - Rainha em (3, 2) na casa branca
 - Rainha em (4, 5) na casa branca
 - Rainha em (5, 8) na casa branca
 - Rainha em (6, 1) na casa branca
 - Rainha em (7, 3) na casa preta
 - Rainha em (8, 6) na casa preta


In [10]:
print("\nTabuleiro (Q para rainha, . vazio):")
for i in model.I:
    linha_str = ""
    for j in model.J:
        if pyo.value(model.x[i, j]) > 0.5:
            linha_str += " Q"
        else:
            linha_str += " ."
    print(linha_str)


Tabuleiro (Q para rainha, . vazio):
 . . . . . . Q .
 . . . Q . . . .
 . Q . . . . . .
 . . . . Q . . .
 . . . . . . . Q
 Q . . . . . . .
 . . Q . . . . .
 . . . . . Q . .
